## Nemo Guardrails - Introduction

> https://www.pinecone.io/learn/nemo-guardrails-intro/

---

In [1]:
import os
import sys
from dotenv import load_dotenv
from langchain_openai import AzureOpenAI, AzureChatOpenAI, AzureOpenAIEmbeddings

In [2]:
load_dotenv("/Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex_tutorials/knowledge_graphs/.env")

True

In [3]:
# print (os.environ['AZURE_API_KEY'])

In [ ]:
print (sys.path)

In [5]:
from nemoguardrails import LLMRails, RailsConfig

/Users/shaunaksen/miniconda3/envs/shaunak_llm/lib/python3.10/site-packages/langchain_community/llms/__init__.py:166: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/Users/shaunaksen/miniconda3/envs/shaunak_llm/lib/python3.10/site-packages/langchain_community/llms/__init__.py:323: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  warn_deprecated(


In [6]:
gpt4_turbo = AzureOpenAI(
        deployment_name="gpt-4-turbo-0125",
        model="gpt-4",
        openai_api_type="azure",
        azure_endpoint="https://ml-dev.openai.azure.com",
        openai_api_key=os.environ['AZURE_API_KEY'],
        openai_api_version="2023-03-15-preview",
        max_retries=2,
        temperature=0,
    )

chat_gpt4 =AzureOpenAI(
        deployment_name="gpt-4-32k",
        model="gpt-4-32k",
        openai_api_type="azure",
        azure_endpoint="https://ml-dev.openai.azure.com",
        openai_api_key=os.environ['AZURE_API_KEY'],
        openai_api_version="2023-03-15-preview",
        max_retries=2,
        temperature=0,
    )

In [7]:
yaml_content = f"""
instructions:
- type: general
  content: /
    Below is a conversation between a personal shopping assistant and
    a user.
models:
  - type: main
    engine: azure
    model: gpt-4
    parameters:
      openai_api_version: '2023-03-15-preview'
      azure_endpoint: "https://ml-dev.openai.azure.com"
      deployment_name: gpt-4-turbo-0125
      api_key: "{os.environ['AZURE_API_KEY']}"
      temperature: 0

"""

In [15]:
colang_content = """
# define niceties
define user express greeting
    "hello"
    "hi"
    "what's up?"

define bot express greeting
    "Hello from your friendly bot :D"

define bot ask how are you
    "How are you doing?"
    "How's it going?"
    "How are you feeling today?"

define bot offer help
    "How can i help you today?"
    "Is there anything else I can help you with?"

# this is a flow called greeting
# [user express greeting, bot express greeting, bot ask how are you] - note that all these have been defined
define flow greeting
    user express greeting
    bot express greeting
    bot ask how are you

# define limits
define user ask politics
    "what are your political beliefs?"
    "thoughts on the president?"
    "left wing"
    "right wing"

define bot answer politics
    "I'm a shopping assistant, I don't like to talk of politics."

# another flow
define flow politics
    user ask politics
    bot answer politics
    bot offer help
"""

In [16]:
# initialize rails config
config = RailsConfig.from_content(
    yaml_content=yaml_content,
    colang_content=colang_content
)

In [17]:
# create rails
rails = LLMRails(config, verbose=False)

In [18]:
res = await rails.generate_async(
    prompt="Hey there!",
)

In [19]:
print (res)

Hello from your friendly bot :D
How are you doing?


The prompt `Hey there!` activated the flow - `greeting` and so we see the responses under that

```
define flow greeting
    user express greeting
    bot express greeting
    bot ask how are you
```

In [20]:
res = await rails.generate_async(prompt="what do you think of the president?")
print(res)

I'm a shopping assistant, I don't like to talk of politics.
How can i help you today?


This prompt activated the flow - `politics` and so we see the responses under that

```
define flow politics
    user ask politics
    bot answer politics
    bot offer help
```

### Colang 101

At the core of NeMo Guardrails is the Colang modeling language. Colang is a mini-language built specifically for developing dialogue flows and safety guardrails for conversational systems.

Colang is simpler, with far fewer constructs than a typical programming language. These constructs also make Colang more flexible than standard programming languages. We describe definitions, and dialogue flows with flexible natural language (using "canonical forms" and "utterances"). Let's take a look at a straightforward Colang file:

```
define user express greeting
        "hello"
        "hi"
        "what's up?"
        
define bot express greeting
        "Hey there!"

define bot ask how are you
        "How are you doing?"

define flow greeting
        user express greeting
        bot express greeting
        bot ask how are you
```

In this Colang script, we have defined the three main types of blocks in Colang. The blocks are user message blocks (define user ...), bot message blocks (define bot ...), and flow blocks (define flow ...).

These represent the primary building blocks from which we can build dialogue flows and guardrails for our chatbots.


__Canonical Forms and Utterances__

We defined three message blocks in our Colang script. The first is a user message block defined by define user express greeting — this structured representation of a message (for both user and bot messages) is known as a canonical form.

Following the canonical form, we have multiple user utterances, which are "examples" of messages that would fit into the defined canonical form. These are "hello", "hi", and "what's up".



In [30]:
colang_content = """
# define niceties
define user express greeting
    "hello"
    "hi"
    "what's up?"

define bot express greeting
    "Hello from your friendly bot :D"

define bot ask how are you
    "How are you doing?"
    "How's it going?"
    "How are you feeling today?"

define bot offer help
    "How can i help you today?"
    "Is there anything else I can help you with?"

# this is a flow called greeting
# [user express greeting, bot express greeting, bot ask how are you] - note that all these have been defined
define flow greeting
    user express greeting
    bot express greeting
    bot ask how are you

# define limits
define user ask politics
    "why doesn't the X party care about Y?"
    "why is Meta lobbying for the X party?"
    "what are your political views?"
    "who should I vote for?"

define bot answer politics
    "I'm a research assistant, I don't like to talk of politics."

define user asks llm
    "what is the llama 2 model?"
    "tell me about Meta's new LLM"
    "what are the differences between Falcon and Llama"

define bot answer llm
    "llama 2 is an open source model by meta"
    "Falcon is much smaller and not as accurate as llama 2"


# another flow
define flow politics
    user ask politics
    bot answer politics
    bot offer help


# another flow
define flow llm
    user asks llm
    bot answer llm
    bot offer help
"""

In [31]:
# initialize rails config
config = RailsConfig.from_content(
    yaml_content=yaml_content,
    colang_content=colang_content
)

In [25]:
# create rails
rails = LLMRails(config, verbose=False)

In [28]:
res = await rails.generate_async(prompt="twhat is the best llm model?")
print(res)

perplexity is the current best llm tool
Is there anything else I can help you with?


Guardrails can flexibly decide which canonical form to use based on a user or bot-message by encoding all utterances specified within the Colang file into a semantic vector space.

![](https://cdn.sanity.io/images/vr8gru94/production/f4c238be0ec0baa83e51ff227d26e8f3df1b7c8b-2895x1710.png)

When given a user/bot-message, we encode it into the same vector space, where we can calculate semantic similarity to identify the most relevant utterances and their respective canonical form.

![](https://cdn.sanity.io/images/vr8gru94/production/f8c83210f38b76eb84a589f8231ce69d8565c112-2293x1534.png)

It's also worth noting that the canonical form itself is encoded into the utterance vector space. Therefore, we could have a functional canonical form without defining its utterances. However, utterances allow us to be more specific in what our canonical form means semantically.


Following this, we defined two bot messages with `define bot express greeting` and `define bot ask how are you`. Both of these use utterances to determine better what these messages refer to.

Finally, we define a dialogue flow with `define flow greeting`. Here we specify what steps are taken if the first user express greeting message is identified as representing some user's input message. In this flow, after the user's message, the chatbot will follow the instructions in the messages `bot express greeting`, and `bot ask how are you`.


